In [ ]:
!pip install catboost
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor, Pool
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/content/train.csv', parse_dates=['doj'])
test = pd.read_csv('/content/test_8gqdJqH (2).csv', parse_dates=['doj'])
transactions = pd.read_csv('/content/transactions.csv', parse_dates=['doj', 'doi'])

In [ ]:
train

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0
...,...,...,...,...
67195,2024-12-31,2,24,800.0
67196,2024-12-31,44,45,1360.0
67197,2024-12-31,47,10,560.0
67198,2024-12-31,38,36,535.0


In [ ]:
test

,route_key,doj,srcid,destid,cumsum_seatcount_x,cumsum_searchcount_x,srcid_region_x,destid_region_x,srcid_tier_x,destid_tier_x,cumsum_seatcount_y,cumsum_searchcount_y,srcid_region_y,destid_region_y,srcid_tier_y,destid_tier_y
0,2025-02-11_46_45,2025-02-11,46,45,38.0,1082.0,Tamil Nadu,Karnataka,Tier 1,Tier 1,38.0,1082.0,Tamil Nadu,Karnataka,Tier 1,Tier 1
1,2025-01-20_17_23,2025-01-20,17,23,0.0,1175.0,East 1,East 1,Tier2,Tier 1,0.0,1175.0,East 1,East 1,Tier2,Tier 1
2,2025-01-08_02_14,2025-01-08,2,14,0.0,370.0,Maharashtra and Goa,Maharashtra and Goa,Tier 1,Tier2,0.0,370.0,Maharashtra and Goa,Maharashtra and Goa,Tier 1,Tier2
3,2025-01-08_08_47,2025-01-08,8,47,0.0,120.0,Andhra Pradesh,Andhra Pradesh,Tier2,Tier 1,0.0,120.0,Andhra Pradesh,Andhra Pradesh,Tier2,Tier 1
4,2025-01-08_09_46,2025-01-08,9,46,39.0,1230.0,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,39.0,1230.0,Tamil Nadu,Tamil Nadu,Tier2,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5895,2025-01-23_46_48,2025-01-23,46,48,126.0,4578.0,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,126.0,4578.0,Tamil Nadu,Tamil Nadu,Tier 1,Tier2
5896,2025-02-21_46_09,2025-02-21,46,9,249.0,11337.0,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,249.0,11337.0,Tamil Nadu,Tamil Nadu,Tier 1,Tier2
5897,2025-01-17_32_19,2025-01-17,32,19,4.0,408.0,Madhya Pradesh,Madhya Pradesh,Tier 1,Tier 1,4.0,408.0,Madhya Pradesh,Madhya Pradesh,Tier 1,Tier 1
5898,2025-01-24_45_03,2025-01-24,45,3,4.0,1132.0,Karnataka,Andhra Pradesh,Tier 1,Tier2,4.0,1132.0,Karnataka,Andhra Pradesh,Tier 1,Tier2


In [ ]:
transactions

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd,diff_days
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30,30
...,...,...,...,...,...,...,...,...,...,...,...,...
2266095,2025-02-28,2025-02-28,2,24,Maharashtra and Goa,Maharashtra and Goa,Tier 1,Tier2,0.0,0.0,0,0
2266096,2025-02-28,2025-02-28,44,45,Tamil Nadu,Karnataka,Tier2,Tier 1,0.0,0.0,0,0
2266097,2025-02-28,2025-02-28,47,10,Andhra Pradesh,Andhra Pradesh,Tier 1,Tier 3,0.0,0.0,0,0
2266098,2025-02-28,2025-02-28,38,36,Rest of North,Delhi,Tier 1,Tier2,0.0,0.0,0,0


In [ ]:
transactions['diff_days'] = (transactions['doj'] - transactions['doi']).dt.days

In [ ]:
t15_data = transactions[transactions['diff_days'] == 15]

In [ ]:
t15_features = t15_data.groupby(['doj', 'srcid', 'destid']).agg({
    'cumsum_seatcount': 'max',
    'cumsum_searchcount': 'max',
    'srcid_region': 'first',
    'destid_region': 'first',
    'srcid_tier': 'first',
    'destid_tier': 'first'
}).reset_index()

In [ ]:
train_df = train.merge(t15_features, on=['doj', 'srcid', 'destid'], how='inner')

In [ ]:
features = ['cumsum_seatcount', 'cumsum_searchcount', 'srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']
X = train_df[features]
y = train_df['final_seatcount']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    X_val[col] = le.transform(X_val[col])
    if col in test.columns:
        test[col] = test[col].apply(lambda x: x if x in le.classes_ else 'unknown')
        le_classes = np.append(le.classes_, 'unknown')
        le.classes_ = le_classes
        test[col] = le.transform(test[col])
    encoders[col] = le

In [ ]:
enc = LabelEncoder()
for col in ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']:
    X_train[col] = enc.fit_transform(X_train[col])
    X_val[col] = enc.transform(X_val[col])

In [ ]:
cat_features = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']


model = CatBoostRegressor(
    iterations=800,
    learning_rate=0.1,
    depth=8,
    loss_function='RMSE',
    verbose=100
)

train_pool = Pool(X_train, y_train, cat_features=cat_features)
val_pool = Pool(X_val, y_val, cat_features=cat_features)
model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50)


test_pool = Pool(test[features], cat_features=cat_features)
test['final_seatcount'] = model.predict(test_pool).round().astype(int)

0:	learn: 1138.7310007	test: 1145.6163547	best: 1145.6163547 (0)	total: 104ms	remaining: 1m 23s
100:	learn: 637.4630281	test: 641.6290012	best: 641.6290012 (100)	total: 5.35s	remaining: 37s
200:	learn: 614.9903229	test: 627.5319399	best: 627.5319399 (200)	total: 12.4s	remaining: 36.9s
300:	learn: 603.4632210	test: 622.6924749	best: 622.6924749 (300)	total: 17.9s	remaining: 29.6s
400:	learn: 595.0933522	test: 619.6242594	best: 619.6242594 (400)	total: 24.6s	remaining: 24.5s
500:	learn: 587.8154877	test: 617.3117168	best: 617.3117168 (500)	total: 30.1s	remaining: 18s
600:	learn: 580.8416415	test: 614.9948046	best: 614.9635735 (599)	total: 37.1s	remaining: 12.3s
700:	learn: 575.3380795	test: 614.4230706	best: 614.4230706 (700)	total: 43.8s	remaining: 6.18s
799:	learn: 570.9783098	test: 613.5400400	best: 613.4596801 (785)	total: 50.6s	remaining: 0us

bestTest = 613.4596801
bestIteration = 785

Shrink model to first 786 iterations.


In [ ]:
val_preds = model.predict(X_val)
rmse = sqrt(mean_squared_error(y_val, val_preds))
print("Validation RMSE:", rmse)

Validation RMSE: 613.4596801398826


In [ ]:
columns_to_drop = ['cumsum_seatcount', 'cumsum_searchcount', 'srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']
test = test.drop(columns=[col for col in columns_to_drop if col in test.columns])


test = test.merge(t15_features, on=['doj', 'srcid', 'destid'], how='left')


import numpy as np

cat_cols = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']
for col in cat_cols:
    le = encoders[col]
    test[col] = test[col].apply(lambda x: x if x in le.classes_ else 'Unknown')
    if 'Unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'Unknown')
    test[col] = le.transform(test[col])


X_test = test[features]
test['final_seatcount'] = model.predict(X_test).round().astype(int)

In [ ]:
submission = test[['route_key', 'final_seatcount']]
submission.to_csv('submission_catboost(103).csv', index=False)
print("✅ Submission file saved as 'submission_catboost.csv'")

✅ Submission file saved as 'submission_catboost.csv'
